# readMe

multi_core_sepsis3_shock_mimiciv_v2.py
定义：脓毒性休克指 脓毒症患者 尽管充分的液体复苏仍存在持续的低血压,需要用升压药维持平均动脉压在 65 mmHg 以上,血乳酸在 2 mmol / L 以上,符合这一标准临床病死率超过 40% 。              
注意： 基于患者已经是脓毒症的情况下


输入数据为 1_get_MIMIC_IV_septic_shock_defined_variable 生成的数据 pickle_mimic_iv_derived_table_v3.1.pkl                
输出数据为 pickle_mimiciv_septic_shock_v4.1.pkl




定义：脓毒性休克指 脓毒症患者 尽管充分的液体复苏仍存在持续的低血压,需要用升压药维持平均动脉压在 65 mmHg 以上,血乳酸在 2 mmol / L 以上,符合这一标准临床病死率超过 40% 。                 
注意： 基于患者已经是脓毒症的情况下                    
液体复苏：vaso前3h 至少1000ml , 去除输液只有1分钟的数据                
mbp: vaso前 一个值 小于等于 65                
lactat: vaso后第一个值 大于等于 2.0           

In [5]:
import os
import pandas as pd
import time
import datetime as dt
import nbimporter

import my_save_load_v2


In [6]:
def splite_data(sepsis3_tem,step=100):
    data = [sepsis3_tem.iloc[i:i+step,:] for i in range(0,sepsis3_tem.shape[0],step) ]
    return data


## 获取 vaso 前3h 输液量，未去除输液只有1分钟的数据
def fluid_3h_pre_vaso(sub_tem_sepsis3,tem_vasoactive_vasopressor,tem_fluids_bolus):
    
    sub_tem_vaso = pd.merge(sub_tem_sepsis3, tem_vasoactive_vasopressor, on='stay_id', how='inner')
    sub_tem_vaso = sub_tem_vaso.loc[sub_tem_vaso.vaso_starttime >= sub_tem_vaso.sofa_time, :]
    sub_tem_vaso.loc[:, 'vaso_3h_pre'] = sub_tem_vaso.vaso_starttime - dt.timedelta(hours=3)

    sub_tem_vaso_fluid = pd.merge(sub_tem_vaso, tem_fluids_bolus, on=['stay_id'], how='inner')

    #去除输液只有1分钟的数据
    #sub_tem_vaso_fluid = sub_tem_vaso_fluid.loc[(sub_tem_vaso_fluid.loc[:, 'fluid_endtime'] - sub_tem_vaso_fluid.loc[:, 'fluid_starttime']) > dt.timedelta(minutes=1),:]

    tem = (sub_tem_vaso_fluid.loc[:, ['vaso_starttime', 'fluid_endtime']].min(axis=1) - sub_tem_vaso_fluid.loc[:, ['vaso_3h_pre','fluid_starttime']].max(axis=1))
    tem = tem / (sub_tem_vaso_fluid.loc[:, 'fluid_endtime'] - sub_tem_vaso_fluid.loc[:, 'fluid_starttime'])

    tem_fluids_bolus_3h = tem.clip(0, tem) * sub_tem_vaso_fluid.loc[:, 'colloid_bolus']
    sub_tem_vaso_fluid.loc[:, 'fluids_bolus_3h'] = tem_fluids_bolus_3h
    sub_tem_vaso_fluid.drop(['fluid_starttime', 'fluid_endtime', 'colloid_bolus'], axis=1, inplace=True)
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.groupby(
        ['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre'], as_index=False).agg(sum)
    return sub_tem_vaso_fluid


# 获取 vaso 前 第一个 mbp值
def fluid_3h_pre_vaso_mbp(sub_tem_vaso_fluid,tem_mbp):
    # 获取 vaso 后 第一个 mbp值
    sub_tem_vaso_fluid_mbp = pd.merge(sub_tem_vaso_fluid,tem_mbp,on=['stay_id'], how='inner')

    #sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[sub_tem_vaso_fluid_mbp.mbp_charttime >= sub_tem_vaso_fluid_mbp.vaso_starttime,:]
    sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[ sub_tem_vaso_fluid_mbp.mbp_charttime <= sub_tem_vaso_fluid_mbp.vaso_starttime, :]

    #sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.sort_values(by=['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre', 'mbp_charttime'])   #时间 按小到大 排序
    sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.sort_values(by=['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre', 'mbp_charttime'],
                                                                ascending=[True,True,True,True,True,False])  # 时间 大到小 排序

    sub_tem_vaso_fluid_mbp=sub_tem_vaso_fluid_mbp.groupby(['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime',
           'vaso_3h_pre', 'fluids_bolus_3h'], as_index=False ).apply(lambda x:x.iloc[0,:])
    return(sub_tem_vaso_fluid_mbp)


# 获取 vaso 后 第一个 lactate 值
def fluid_3h_pre_vaso_mbp_lactate(sub_tem_vaso_fluid_mbp,tem_lactate):
    # 获取 vaso 后 第一个 lactate 值
    sub_tem_vaso_fluid_mbp_lactate = pd.merge(sub_tem_vaso_fluid_mbp, tem_lactate, on=['stay_id'], how='inner')
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.loc[
                                     sub_tem_vaso_fluid_mbp_lactate.lactate_charttime >= sub_tem_vaso_fluid_mbp_lactate.vaso_starttime,
                                     :]
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.sort_values(
        by=['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre',
            'mbp_charttime', 'lactate_charttime'])
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.groupby(
        ['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime',
         'vaso_3h_pre', 'fluids_bolus_3h', 'mbp_charttime', 'mbp'], as_index=False).apply(lambda x: x.iloc[0, :])
    return(sub_tem_vaso_fluid_mbp_lactate)


def main_code(i,num,t0,tem_sepsis3_split,tem_vasoactive_vasopressor,tem_fluids_bolus,tem_mbp,tem_lactate):
    sub_tem_sepsis3=tem_sepsis3_split[i]
    sub_tem_vaso_fluid = fluid_3h_pre_vaso(sub_tem_sepsis3,tem_vasoactive_vasopressor,tem_fluids_bolus)
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.loc[sub_tem_vaso_fluid.fluids_bolus_3h >= 1000]

    sub_tem_vaso_fluid_mbp = fluid_3h_pre_vaso_mbp(sub_tem_vaso_fluid,tem_mbp)
    #sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[sub_tem_vaso_fluid_mbp.mbp >= 65,:]
    sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[sub_tem_vaso_fluid_mbp.mbp <= 65,:]

    sub_tem_vaso_fluid_mbp_lactate = fluid_3h_pre_vaso_mbp_lactate(sub_tem_vaso_fluid_mbp,tem_lactate)
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.loc[sub_tem_vaso_fluid_mbp_lactate.lactate >= 2,:]

    shock = sub_tem_vaso_fluid_mbp_lactate.sort_values(by=['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre'])
    shock = shock.groupby(['stay_id', 'sofa_time'], as_index=False).apply(lambda x:x.iloc[0,:])

    print(f'complete {i} / {num} ; spend time : { time.time() - t0 :.4f} s' )

    return(shock)




# 导入数据


In [7]:
#path = r'I:\MIMIC_chuli\MIMIC_IV'
path = r'E:\MIMIC_chuli\MIMIC_IV3'
os.chdir(path)

my_save_load_v2.my_load('pickle_mimic_iv_derived_table_v3.1.pkl', globals())


# 计算各个指标

In [8]:
sepsis3 = sepsis3_original.copy()
sepsis3 = sepsis3.sort_values(by='stay_id')
tem_sepsis3 = sepsis3.loc[:,['stay_id','sofa_time']]

tem_vasoactive_vasopressor = vasoactive_vasopressor.loc[:,['stay_id','starttime','endtime']]
tem_vasoactive_vasopressor.columns = ['stay_id','vaso_starttime','vaso_endtime']

tem_fluids_bolus = fluids_bolus.loc[:,['stay_id','starttime','endtime','colloid_bolus']]
tem_fluids_bolus.columns = ['stay_id','fluid_starttime','fluid_endtime','colloid_bolus']

tem_mbp=shock_define_mbp.copy()
tem_mbp.drop(columns='subject_id',inplace=True)
tem_mbp.columns = ['stay_id', 'mbp_charttime', 'mbp']

'''
shock_define_lactate2 = pd.merge(icustays.loc[:,['subject_id','stay_id','intime', 'outtime']],
                                 shock_define_lactate.loc[:,['subject_id', 'charttime', 'lactate']],
                                 on=['subject_id'],how='inner')
shock_define_lactate2 = shock_define_lactate2.loc[(shock_define_lactate2.charttime >= shock_define_lactate2.intime) & (shock_define_lactate2.charttime <= shock_define_lactate2.outtime),:]
shock_define_lactate2 = shock_define_lactate2.loc[:,['subject_id', 'stay_id', 'charttime', 'lactate']]
'''

tem_lactate = shock_define_lactate2.copy()
tem_lactate.drop(columns='subject_id',inplace=True)
tem_lactate.columns = ['stay_id', 'lactate_charttime', 'lactate']


tem_sepsis3_split = splite_data(tem_sepsis3,step=1000)

num = tem_sepsis3_split.__len__()
t0=time.time()

septic_shock = pd.DataFrame()

for i in range(num):
    shock = main_code(i,num,t0,tem_sepsis3_split,tem_vasoactive_vasopressor,tem_fluids_bolus,tem_mbp,tem_lactate)
    septic_shock = pd.concat([septic_shock,shock],axis=0)


septic_shock = septic_shock.loc[:,['stay_id','fluids_bolus_3h','vaso_starttime','mbp','lactate']]
septic_shock.columns =  ['stay_id', 'fluid_amound_vaso_3Hour_ago', 'fluid_amound_vaso_time', 'MBP', 'Lactate']

complete 0 / 32 ; spend time : 8.2614 s
complete 1 / 32 ; spend time : 16.1856 s


C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: divide by zero encountered in true_divide
  return other / self._data
C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: invalid value encountered in true_divide
  return other / self._data


complete 2 / 32 ; spend time : 24.3600 s
complete 3 / 32 ; spend time : 31.2863 s


C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: divide by zero encountered in true_divide
  return other / self._data
C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: invalid value encountered in true_divide
  return other / self._data


complete 4 / 32 ; spend time : 39.7807 s


C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: divide by zero encountered in true_divide
  return other / self._data
C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: invalid value encountered in true_divide
  return other / self._data


complete 5 / 32 ; spend time : 47.1920 s
complete 6 / 32 ; spend time : 53.8547 s
complete 7 / 32 ; spend time : 62.4607 s
complete 8 / 32 ; spend time : 69.5534 s
complete 9 / 32 ; spend time : 76.2525 s
complete 10 / 32 ; spend time : 83.7183 s
complete 11 / 32 ; spend time : 91.7777 s
complete 12 / 32 ; spend time : 99.2425 s
complete 13 / 32 ; spend time : 106.5123 s
complete 14 / 32 ; spend time : 114.2546 s
complete 15 / 32 ; spend time : 121.1301 s
complete 16 / 32 ; spend time : 128.1179 s
complete 17 / 32 ; spend time : 135.5081 s
complete 18 / 32 ; spend time : 141.4417 s
complete 19 / 32 ; spend time : 149.4596 s
complete 20 / 32 ; spend time : 157.6185 s
complete 21 / 32 ; spend time : 165.8859 s
complete 22 / 32 ; spend time : 175.1140 s
complete 23 / 32 ; spend time : 184.0766 s
complete 24 / 32 ; spend time : 191.3482 s
complete 25 / 32 ; spend time : 201.0114 s
complete 26 / 32 ; spend time : 207.9911 s
complete 27 / 32 ; spend time : 216.4961 s
complete 28 / 32 ; spend

C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: divide by zero encountered in true_divide
  return other / self._data
C:\Users\LGH\.conda\envs\jupy\lib\site-packages\pandas\core\arrays\timedeltas.py:598: RuntimeWarning: invalid value encountered in true_divide
  return other / self._data


complete 29 / 32 ; spend time : 232.1923 s
complete 30 / 32 ; spend time : 241.0388 s
complete 31 / 32 ; spend time : 245.0086 s


In [12]:
septic_shock.iloc[:20,:]

,stay_id,fluid_amound_vaso_3Hour_ago,fluid_amound_vaso_time,MBP,Lactate
0,30003598,1097.188456,2189-04-15 20:03:00,64.0,2.2
1,30003749,1324.030290,2120-11-05 17:15:00,59.0,12.8
2,30005707,1123.958894,2144-01-07 01:15:00,63.0,2.9
3,30009123,1166.868242,2188-04-28 14:30:00,65.0,2.3
4,30009505,1256.159954,2155-07-05 18:08:00,65.0,5.7
5,30009597,1460.461538,2175-10-04 01:33:00,64.0,6.4
6,30014019,1356.907239,2119-07-13 13:48:00,60.0,9.9
7,30020307,4312.440801,2131-08-30 00:00:00,54.0,2.2
8,30022281,5180.000000,2123-04-04 20:00:00,61.0,2.1
9,30025720,2236.337897,2164-09-25 12:26:00,54.0,2.5


# 保存数据

In [13]:
###### save
readMe_shock_defined= """
定义：脓毒性休克指 脓毒症患者 尽管充分的液体复苏仍存在持续的低血压,需要用升压药维持平均动脉压在 65 mmHg 以上,血乳酸在 2 mmol / L 以上,符合这一标准临床病死率超过 40% 。
注意： 基于患者已经是脓毒症的情况下
液体复苏：vaso前3h 至少1000ml , 去除输液只有1分钟的数据
mbp: vaso 前第一个值 小于等于 65
lactat: vaso后第一个值 大于等于 2.0
"""

out_name = 'pickle_mimiciv_septic_shock_v4.1.pkl' # 保留1分钟，MBP 取 vasso前一个 小于65


l = ['readMe_shock_defined','septic_shock', 'age', 'fluids_bolus', 'fluids_colloid_bolus', 'fluids_crystalloid_bolus', 'icustays',
     'sepsis3_original', 'shock_define_lactate', 'shock_define_mbp', 'vasoactive_vasopressor']
my_save_load_v2.my_save(List=l, filename=out_name, Global=globals())

